In [1]:
!pip install google-cloud-bigquery
!pip install db-dtypes
!pip install tqdm
!pip install pyspark
!pip install pandas

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
import requests
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import os
from pyspark.sql import Row
from tqdm import tqdm


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print(os.environ["JAVA_HOME"])

spark = SparkSession.builder.appName("recommend") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog",
                "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.executor.memory", "64g") \
        .config("spark.driver.memory", "64g") \
        .config('spark.kryoserializer.buffer.max', '1g') \
        .getOrCreate()


/usr/lib/jvm/java-11-openjdk-amd64


ConnectionRefusedError: [Errno 111] Connection refused

In [5]:
likes_df = spark.read.csv('likes.csv', inferSchema=True, header=True)

likes_df.show(10)

ConnectionRefusedError: [Errno 111] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/bryan/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bryan/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/bryan/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [3]:
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.ml.feature import StringIndexer, StringIndexerModel

# Rename the columns to match the names expected by ALS
ratings_df = likes_df.selectExpr("actioned_by_profile_id as userId", "publication_id as postId", "reaction as rating")

# Create a StringIndexer for userId
indexer_user = StringIndexer(inputCol="userId", outputCol="userIdIndex")

# Fit the StringIndexer to the data and transform it
indexer_model_user = indexer_user.fit(ratings_df)
ratings_df = indexer_model_user.transform(ratings_df)

# Save the model
indexer_model_user.save("user_model")

# Create a StringIndexer for postId
indexer_post = StringIndexer(inputCol="postId", outputCol="postIdIndex")

# Fit the StringIndexer to the data and transform it
indexer_model_post = indexer_post.fit(ratings_df)
ratings_df = indexer_model_post.transform(ratings_df)

# Save the model
indexer_model_post.save("post_model")

ratings_df = ratings_df.filter(ratings_df.userId.isNotNull())

# Map "UPVOTE" to 1 and "DOWNVOTE" to -1
ratings_df = ratings_df.withColumn("rating", when(col("rating") == "UPVOTE", 1).otherwise(-1))

ratings_df.show(5)


23/10/21 18:36:08 WARN TaskSetManager: Stage 6 contains a task of very large size (1509 KiB). The maximum recommended task size is 1000 KiB.
23/10/21 18:36:57 WARN TaskSetManager: Stage 13 contains a task of very large size (115869 KiB). The maximum recommended task size is 1000 KiB.
23/10/21 18:37:15 WARN DAGScheduler: Broadcasting large task binary with size 169.0 MiB


+--------+--------------------+------+-----------+-----------+
|  userId|              postId|rating|userIdIndex|postIdIndex|
+--------+--------------------+------+-----------+-----------+
|0x015779|       0x011292-0x5c|     1|    20622.0|   270187.0|
|0x0157dd|         0x4fa0-0x91|     1|    10721.0|   101426.0|
|0x015a3c|         0x0d-0x03ae|     1|    14634.0|      146.0|
|0x015a87|0xd978-0x0126-DA-...|     1|     1106.0|    62859.0|
|0x015ab7|       0x01ce98-0x01|     1|    21079.0|      951.0|
+--------+--------------------+------+-----------+-----------+
only showing top 5 rows



In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Assume you have a DataFrame `ratings_df` with columns 'userId', 'postId', and 'rating'
# Split the data into training and test sets
(training, test) = ratings_df.randomSplit([0.8, 0.2])
# Create an ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="userIdIndex", itemCol="postIdIndex", ratingCol="rating",
          coldStartStrategy="drop")

# Train the ALS model
model = als.fit(training)

# Make predictions on the test data
predictions = model.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

23/10/21 18:38:59 WARN DAGScheduler: Broadcasting large task binary with size 169.0 MiB
23/10/21 18:39:19 WARN DAGScheduler: Broadcasting large task binary with size 169.0 MiB
23/10/21 18:39:45 WARN DAGScheduler: Broadcasting large task binary with size 169.0 MiB
23/10/21 18:40:19 WARN DAGScheduler: Broadcasting large task binary with size 169.1 MiB
23/10/21 18:40:40 WARN DAGScheduler: Broadcasting large task binary with size 169.0 MiB
23/10/21 18:40:51 ERROR Executor: Exception in task 9.0 in stage 22.0 (TID 310)]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream

[370.729s][warning][gc,alloc] Executor task launch worker for task 9.0 in stage 22.0 (TID 310): Retried waiting for GCLocker too often allocating 8388611 words


23/10/21 18:40:54 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 9.0 in stage 22.0 (TID 310),5,main]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2134)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInp

Py4JJavaError: An error occurred while calling o175.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 22.0 failed 1 times, most recent failure: Lost task 9.0 in stage 22.0 (TID 310) (155.246.81.32 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2134)
	at java.base/java.io.ObjectInputStream.readObject0(Objec

23/10/21 18:40:55 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 6.0 in stage 22.0 (TID 307)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:146)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
23/10/21

In [6]:
# Generate top 10 post recommendations for each user
userRecs = model.recommendForAllUsers(10)

userRecs.show()

# Convert the recommendations array to a string
userRecs = userRecs.withColumn("recommendations", col("recommendations").cast("string"))

# Save the DataFrame to a CSV file
userRecs.write.csv("user_recommendations.csv")

23/10/18 23:37:33 WARN DAGScheduler: Broadcasting large task binary with size 169.1 MiB
23/10/18 23:39:24 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:28 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:31 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:35 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:38 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:41 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:44 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:48 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:48 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.


[850.191s][warning][gc,alloc] Executor task launch worker for task 3.0 in stage 94.0 (TID 676): Retried waiting for GCLocker too often allocating 33554435 words
[850.463s][warning][gc,alloc] Executor task launch worker for task 43.0 in stage 94.0 (TID 716): Retried waiting for GCLocker too often allocating 33554435 words


23/10/18 23:39:48 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.


[850.673s][warning][gc,alloc] Executor task launch worker for task 44.0 in stage 94.0 (TID 717): Retried waiting for GCLocker too often allocating 33554435 words
[850.787s][warning][gc,alloc] Executor task launch worker for task 31.0 in stage 94.0 (TID 704): Retried waiting for GCLocker too often allocating 33554435 words


23/10/18 23:39:48 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:48 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.


...

[853.998s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 94.0 (TID 678): Retried waiting for GCLocker too often allocating 33554435 words
[854.045s][warning][gc,alloc] Executor task launch worker for task 14.0 in stage 94.0 (TID 687): Retried waiting for GCLocker too often allocating 33554435 words
[854.086s][warning][gc,alloc] Executor task launch worker for task 29.0 in stage 94.0 (TID 702): Retried waiting for GCLocker too often allocating 33554435 words
[854.122s][warning][gc,alloc] Executor task launch worker for task 33.0 in stage 94.0 (TID 706): Retried waiting for GCLocker too often allocating 33554435 words
[854.174s][warning][gc,alloc] Executor task launch worker for task 41.0 in stage 94.0 (TID 714): Retried waiting for GCLocker too often allocating 33554435 words


23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.


[854.266s][warning][gc,alloc] Executor task launch worker for task 40.0 in stage 94.0 (TID 713): Retried waiting for GCLocker too often allocating 33554435 words
[854.325s][warning][gc,alloc] Executor task launch worker for task 27.0 in stage 94.0 (TID 700): Retried waiting for GCLocker too often allocating 33554435 words


23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:39:52 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:41:50 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:41:57 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:42:01 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:42:04 WARN TaskMemoryManager: Failed to allocate a page (268435456 bytes), try again.
23/10/18 23:42:33 WARN TaskMemoryManager: Failed to allocate a page (536870912 bytes), try again.
23/10/18 23:42:36 WARN TaskMemoryManager: Failed to allocate a page (377615988 bytes), try again.
23/10/18 23:42:36 ERROR Executor: Exception in task 56.0 in stage 94.0 (TID 729)
org.apache.spark.memory.SparkOutOfMemoryError: [UNABLE_TO_ACQUIRE_MEMORY] Unable to acquire 65536 bytes of memory, got 0.
	at org.apach

23/10/18 23:42:37 WARN TaskSetManager: Lost task 91.0 in stage 94.0 (TID 764) (155.246.81.32 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 56 in stage 94.0 failed 1 times, most recent failure: Lost task 56.0 in stage 94.0 (TID 729) (155.246.81.32 executor driver): org.apache.spark.memory.SparkOutOfMemoryError: [UNABLE_TO_ACQUIRE_MEMORY] Unable to acquire 65536 bytes of memory, got 0.
	at org.apache.spark.errors.SparkCoreErrors$.outOfMemoryError(SparkCoreErrors.scala:467)
	at org.apache.spark.errors.SparkCoreErrors.outOfMemoryError(SparkCoreErrors.scala)
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:157)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:98)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.growPointerArrayIfNecessary(UnsafeExternalSorter.java:415)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.allocateMemoryForRecordIfNecessary(UnsafeEx

In [7]:
# Assume `model` is the trained model
model.save("weights")

23/10/18 23:42:41 WARN TaskSetManager: Lost task 74.0 in stage 94.0 (TID 747) (155.246.81.32 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 56 in stage 94.0 failed 1 times, most recent failure: Lost task 56.0 in stage 94.0 (TID 729) (155.246.81.32 executor driver): org.apache.spark.memory.SparkOutOfMemoryError: [UNABLE_TO_ACQUIRE_MEMORY] Unable to acquire 65536 bytes of memory, got 0.
	at org.apache.spark.errors.SparkCoreErrors$.outOfMemoryError(SparkCoreErrors.scala:467)
	at org.apache.spark.errors.SparkCoreErrors.outOfMemoryError(SparkCoreErrors.scala)
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:157)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:98)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.growPointerArrayIfNecessary(UnsafeExternalSorter.java:415)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.allocateMemoryForRecordIfNecessary(UnsafeEx